In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [177]:
data_train = pd.read_csv('input/train.csv')

print(data_train.shape)
data_train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


去除“无用”的列，标称型数据转换。

In [182]:
m = {'Sex': {'male': 0, 'female': 1}, 'Embarked': {'S': 0, 'C': 1, 'Q': 2}}

data_train_basic = data_train.drop(['Name', 'Ticket', 'Cabin'], axis=1)
data_train_basic = data_train_basic.replace(m)

data_train_basic = data_train_basic.dropna(axis=0, how='any')

print(data_train_basic.shape)
data_train_basic.head()

(712, 9)


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,0.0
1,2,1,1,1,38.0,1,0,71.2833,1.0
2,3,1,3,1,26.0,0,0,7.9250,0.0
3,4,1,1,1,35.0,1,0,53.1000,0.0
4,5,0,3,0,35.0,0,0,8.0500,0.0


In [184]:
X = data_train_basic.drop('Survived', axis=1).values
y = data_train_basic.loc[:, 'Survived'].values

print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (712, 8)
y shape:  (712,)


逻辑回归，通过GridSearchCV选择参数

In [202]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

logistic = LogisticRegression()
estimator = GridSearchCV(logistic, {'C':[1, 10, 15, 20, 50]})
estimator.fit(X, y)

print('GridSearchCV result:')
for key in estimator.cv_results_:
    print(key, ': ', estimator.cv_results_[key], sep='')

print()
print('best estimator:')
print(estimator.best_estimator_)

GridSearchCV result:
split0_test_score: [ 0.79411765  0.79831933  0.80252101  0.79831933  0.80252101]
split1_test_score: [ 0.78481013  0.78902954  0.78902954  0.78902954  0.78902954]
split2_test_score: [ 0.78059072  0.78481013  0.78481013  0.78481013  0.78059072]
mean_test_score: [ 0.78651685  0.79073034  0.79213483  0.79073034  0.79073034]
std_test_score: [ 0.00565429  0.00564632  0.00755824  0.00564632  0.00903634]
rank_test_score: [5 2 1 2 2]
split0_train_score: [ 0.80590717  0.80590717  0.80590717  0.80801688  0.81012658]
split1_train_score: [ 0.78947368  0.80210526  0.80210526  0.80421053  0.80421053]
split2_train_score: [ 0.79157895  0.80210526  0.8         0.8         0.8       ]
mean_train_score: [ 0.79565327  0.80337257  0.80267081  0.8040758   0.80477904]
std_train_score: [ 0.00730137  0.00179224  0.00244453  0.00327426  0.00415366]
mean_fit_time: [ 0.00233324  0.00283329  0.00199993  0.00183336  0.00166678]
std_fit_time: [  8.49884092e-04   6.23543135e-04   1.12391596e-07   

交叉验证cross_val_score

In [217]:
from sklearn.model_selection import cross_val_score

logistic = LogisticRegression(C=15)
score = cross_val_score(logistic, X, y, cv=5)
score

array([ 0.74825175,  0.82517483,  0.76923077,  0.76056338,  0.79432624])

决策树

In [251]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
estimator = GridSearchCV(dt, {'criterion':['gini', 'entropy'], 'max_features':['sqrt', 'log2'], 'random_state':[None]})
estimator.fit(X, y)

print('GridSearchCV result:')
for key in estimator.cv_results_:
    print(key, ': ', estimator.cv_results_[key], sep='')

print()
print('best estimator:')
print(estimator.best_estimator_)

GridSearchCV result:
split0_test_score: [ 0.58823529  0.60504202  0.59663866  0.68487395]
split1_test_score: [ 0.69198312  0.75949367  0.65400844  0.72995781]
split2_test_score: [ 0.73839662  0.77637131  0.75949367  0.67932489]
mean_test_score: [ 0.67275281  0.71348315  0.66994382  0.69803371]
std_test_score: [ 0.06281088  0.07714893  0.06745427  0.02266356]
rank_test_score: [3 1 4 2]
split0_train_score: [ 1.  1.  1.  1.]
split1_train_score: [ 1.  1.  1.  1.]
split2_train_score: [ 1.  1.  1.  1.]
mean_train_score: [ 1.  1.  1.  1.]
std_train_score: [ 0.  0.  0.  0.]
mean_fit_time: [ 0.00149989  0.00149989  0.00116674  0.00100001]
std_fit_time: [  0.00000000e+00   4.08218701e-04   2.35572866e-04   1.12391596e-07]
mean_score_time: [ 0.00050004  0.00049996  0.00033331  0.00033339]
std_score_time: [  1.12391596e-07   0.00000000e+00   2.35685177e-04   2.35741393e-04]
param_criterion: ['gini' 'gini' 'entropy' 'entropy']
param_max_features: ['sqrt' 'log2' 'sqrt' 'log2']
param_random_state: [N

In [252]:
from sklearn.model_selection import cross_val_score

dt = DecisionTreeClassifier(criterion='gini', max_features='log2')
score = cross_val_score(dt, X, y, cv=5)
score

array([ 0.6013986 ,  0.74125874,  0.66433566,  0.78873239,  0.80141844])

SVM

In [308]:
from sklearn.svm import LinearSVC

svc = LinearSVC()
estimator = GridSearchCV(svc, {'C':[0.1, 1, 10]})
estimator.fit(X, y)

print('GridSearchCV result:')
for key in estimator.cv_results_:
    print(key, ': ', estimator.cv_results_[key], sep='')

print()
print('best estimator:')
print(estimator.best_estimator_)

GridSearchCV result:
split0_test_score: [ 0.64285714  0.71008403  0.69747899]
split1_test_score: [ 0.6371308   0.40506329  0.78059072]
split2_test_score: [ 0.40506329  0.60759494  0.73839662]
mean_test_score: [ 0.56179775  0.5744382   0.73876404]
std_test_score: [ 0.11073597  0.12675024  0.03394305]
rank_test_score: [3 2 1]
split0_train_score: [ 0.62447257  0.48945148  0.54219409]
split1_train_score: [ 0.64421053  0.50315789  0.78947368]
split2_train_score: [ 0.52842105  0.68        0.78526316]
mean_train_score: [ 0.59903472  0.55753646  0.70564364]
std_train_score: [ 0.05057743  0.0867754   0.11558907]
mean_fit_time: [ 0.02783895  0.02583838  0.02600503]
std_fit_time: [  1.92968122e-03   2.35853784e-04   1.94667955e-07]
mean_score_time: [ 0.00050004  0.00016673  0.00033347]
std_score_time: [  1.12391596e-07   2.35797568e-04   2.35797649e-04]
param_C: [0.1 1 10]
params: ({'C': 0.1}, {'C': 1}, {'C': 10})

best estimator:
LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,


In [309]:
from sklearn.model_selection import cross_val_score

svc = LinearSVC(C=10)
score = cross_val_score(svc, X, y, cv=5)
score

array([ 0.66433566,  0.75524476,  0.59440559,  0.75352113,  0.43262411])

In [312]:
from sklearn.svm import SVC

svc = SVC()
estimator = GridSearchCV(svc, {'C':[0.1, 1, 10]})
estimator.fit(X, y)

print('GridSearchCV result:')
for key in estimator.cv_results_:
    print(key, ': ', estimator.cv_results_[key], sep='')

print()
print('best estimator:')
print(estimator.best_estimator_)

GridSearchCV result:
split0_test_score: [ 0.59663866  0.59663866  0.59663866]
split1_test_score: [ 0.59493671  0.58649789  0.57805907]
split2_test_score: [ 0.59493671  0.59493671  0.59493671]
mean_test_score: [ 0.59550562  0.59269663  0.58988764]
std_test_score: [ 0.00080287  0.00443338  0.00838411]
rank_test_score: [1 2 3]
split0_train_score: [ 0.59493671  1.          1.        ]
split1_train_score: [ 0.59578947  1.          1.        ]
split2_train_score: [ 0.59578947  1.          1.        ]
mean_train_score: [ 0.59550522  1.          1.        ]
std_train_score: [ 0.000402  0.        0.      ]
mean_fit_time: [ 0.01700338  0.01533628  0.01683672]
std_fit_time: [ 0.00282901  0.00023585  0.00047148]
mean_score_time: [ 0.00683475  0.0065012   0.00650136]
std_score_time: [  4.71538951e-04   1.94667955e-07   1.12391596e-07]
param_C: [0.1 1 10]
params: ({'C': 0.1}, {'C': 1}, {'C': 10})

best estimator:
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None

In [313]:
from sklearn.model_selection import cross_val_score

svc = SVC(C=0.1)
score = cross_val_score(svc, X, y, cv=5)
score

array([ 0.59440559,  0.59440559,  0.59440559,  0.59859155,  0.59574468])